In [1]:
import numpy as nmp
import pandas as pnd
import matplotlib.pyplot as plt

import pymc3 as pmc

import clonosGP as cln

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
DATA = pnd.read_csv('data/cll_Rincon_2019_patient1.csv')
METRICS = pnd.read_csv('results/cll_Rincon_2019_patient1.csv')

In [4]:
nmp.random.seed(42)
pmc.tt_rng(42);

RES1 = cln.infer(DATA, 
                 model_args={'K': 20, 'prior': 'Flat', 'cov': 'Mat32', 'lik': 'Bin', 'threshold': 0.015},
                 pymc3_args={'niters': 40000, 'method': 'advi', 'flow': 'scale-loc', 'learning_rate': 1e-2, 'random_seed': 42})

INFO:clonosGP:No PURITY column in the data. Assuming all samples have purity 100%.
INFO:clonosGP:No CNn column in the data. Assuming germline is diploid over all provided loci.
INFO:clonosGP:No CNt column in the data. Assuming all tumour samples are diploid over all provided loci.
INFO:clonosGP:No CNm column in the data. Multiplicity values will be approximated.
Average Loss = 9,358.8: 100%|██████████| 40000/40000 [00:29<00:00, 1365.68it/s] 
Finished [100%]: Average Loss = 9,358.8
INFO:pymc3.variational.inference:Finished [100%]: Average Loss = 9,358.8
INFO:clonosGP:Calculating posterior cluster weights and centres.
INFO:clonosGP:Calculating posterior CCF values.
INFO:clonosGP:Calculating posterior predictive distribution.
INFO:clonosGP:Finished.


In [5]:
nmp.random.seed(42)
pmc.tt_rng(42);

RES2 = cln.infer(DATA, 
                 model_args={'K': 20, 'prior': 'GP0', 'cov': 'Mat32', 'lik': 'Bin', 'threshold': 0.015}, 
                 pymc3_args={'niters': 40000, 'method': 'advi', 'flow': 'scale-loc', 'learning_rate': 1e-2, 'random_seed': 42})

INFO:clonosGP:No PURITY column in the data. Assuming all samples have purity 100%.
INFO:clonosGP:No CNn column in the data. Assuming germline is diploid over all provided loci.
INFO:clonosGP:No CNt column in the data. Assuming all tumour samples are diploid over all provided loci.
INFO:clonosGP:No CNm column in the data. Multiplicity values will be approximated.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
  0%|          | 0/40000 [00:00<?, ?it/s]/home/dimitris/VENVS/py37/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
Average Loss = 6,170.1: 100%|█

In [6]:
data1, centres1 = RES1['data'], RES1['centres'] 
data2, centres2, centres_gp = RES2['data'], RES2['centres'], RES2['centres_gp']

In [7]:
%load_ext rpy2.ipython
%R library(tidyverse)
%R library(patchwork)


✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



array(['patchwork', 'forcats', 'stringr', 'dplyr', 'purrr', 'readr',
       'tidyr', 'tibble', 'ggplot2', 'tidyverse', 'tools', 'stats',
       'graphics', 'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

In [11]:
%%R -i data1,data2,centres1,centres2,centres_gp,METRICS -w 10 -h 10 --units in

df2 = data2 %>% filter(CLUSTERID != 'uncertain')
cids2 = df2 %>% pull(CLUSTERID) %>% unique()
colors = colorRampPalette(RColorBrewer::brewer.pal(8, 'Set2'))(length(cids2))

gg1 =
    df2 %>%
    ggplot() +
    geom_line(aes(x = TIME2, y = VAF, group = MUTID, color = CLUSTERID)) +
    scale_x_continuous(breaks = unique(df2$TIME2), labels = unique(df2$SAMPLEID)) +
    scale_color_manual(values = colors) +
    labs(x = NULL, y = 'variant allele fraction') +
    theme_bw() +    
    theme(legend.position = 'none',
          axis.text.x = element_blank())

ctrs2 = centres2 %>% filter(CLUSTERID %in% cids2) %>% mutate(CLUSTERID = as.character(CLUSTERID))
ctrs_gp = centres_gp %>% filter(CLUSTERID %in% cids2) %>% mutate(CLUSTERID = as.character(CLUSTERID))
gg2 =
    ggplot() +
    geom_ribbon(aes(x = TIME, ymin = PHI_LO, ymax = PHI_HI, fill = CLUSTERID), data = ctrs_gp, alpha = 0.5) +
    geom_line(aes(x = TIME, y = PHI, color = CLUSTERID), data = ctrs_gp) +
    geom_point(aes(x = TIME2, y = PHI, color = CLUSTERID), data = ctrs2) +
    scale_x_continuous(breaks = unique(df2$TIME2), labels = unique(df2$SAMPLEID)) +
    scale_color_manual(values = colors) +
    scale_fill_manual(values = colors) +
    labs(x = NULL, y = 'cancer cell fraction') +
    theme_bw() +
    theme(legend.position = 'none',
          axis.text.x = element_blank())

df1 = data1 %>% filter(CLUSTERID != 'uncertain')     
cids1 = df1 %>% pull(CLUSTERID) %>% unique()
ctrs1 = centres1 %>% mutate(CLUSTERID = as.character(CLUSTERID)) %>% filter(CLUSTERID %in% cids1)
gg3 =
    ctrs1 %>%
    ggplot() +
    geom_line(aes(x = TIME2, y = PHI, group = CLUSTERID), linetype = 'dashed') +
    geom_linerange(aes(x = TIME2, ymin = PHI_LO, ymax=PHI_HI, group = CLUSTERID)) +    
    geom_point(aes(x = TIME2, y = PHI, group = CLUSTERID)) +
    scale_x_continuous(breaks = unique(df1$TIME2), labels = unique(df1$SAMPLEID)) +
    labs(x = 'sample', y = 'cancer cell fraction') +
    theme_bw() +
    theme(legend.position = 'none',
          axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5))
    
metrics = 
    METRICS %>% 
    filter(LIK == 'Bin', METRIC == 'LOSS') %>% 
    mutate(LABEL = if_else(PRIOR == 'Flat', 'Flat', str_c(PRIOR, COV, sep='-'))) %>%
    mutate(LABEL = factor(LABEL, levels = .$LABEL))

med = metrics %>% filter(LABEL == 'Flat') %>% pull(MEDIAN)

gg4 =
    metrics %>%
    ggplot() +
    geom_hline(yintercept = -med, linetype = 'dashed') +        
    geom_linerange(aes(x = LABEL, ymin = -HIGH, ymax=-LOW)) +    
    geom_point(aes(x = LABEL, y = -MEDIAN)) +
    labs(x = NULL, y = 'evidence lower bound') +
    theme_bw() +
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5))
    
gg1 / gg2 / gg3 / gg4 +
    plot_annotation(tag_levels = 'A')
    
# ggsave('tmp.pdf')

In [9]:
data1.CLUSTERID.unique(), data2.CLUSTERID.unique() 

(array(['6', '16', 'uncertain', '11', '18', '7'], dtype=object),
 array(['3', '16', '17', '10', '15', '8', '20', '4', '9'], dtype=object))

In [10]:
data2.query('CLUSTERID == "uncertain"').groupby('CLUSTERID').count()

,SAMPLEID,PURITY,TIME,TIME2,MUTID,r,R,CNn,CNt,CNm,VAF,VAF0,CCF_,CCF,CCF_HI,CCF_LO,PPD,PPD_HI,PPD_LO
CLUSTERID,,,,,,,,,,,,,,,,,,,
